<a href="https://colab.research.google.com/github/akashravichandran/colab-experiments/blob/master/english_to_tamil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! wget https://www.manythings.org/anki/tam-eng.zip

--2020-10-31 14:39:08--  https://www.manythings.org/anki/tam-eng.zip
Resolving www.manythings.org (www.manythings.org)... 172.67.173.198, 104.24.109.196, 104.24.108.196, ...
Connecting to www.manythings.org (www.manythings.org)|172.67.173.198|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9823 (9.6K) [application/zip]
Saving to: ‘tam-eng.zip’

tam-eng.zip         100%[===================>]   9.59K  --.-KB/s    in 0s      

2020-10-31 14:39:08 (82.3 MB/s) - ‘tam-eng.zip’ saved [9823/9823]



In [3]:
! unzip tam-eng.zip

Archive:  tam-eng.zip
  inflating: _about.txt              
  inflating: tam.txt                 


In [5]:
import subprocess
import argparse
import os

import pandas as pd

# parser = argparse.ArgumentParser()
# parser.add_argument('-s', help='ISO 639-3 code for a source language')
# parser.add_argument('-t', help='ISO 639-3 code for a target language')
# parser.add_argument('-data_dir', default='./data', help='Data directory')


if __name__ == '__main__':
    # args = parser.parse_args()
    source_lang = 'eng'
    target_lang = 'tam'
    data_dir = 'data/'

    # download if sentences are not here
    sentences_path = os.path.join(data_dir, 'sentences.csv')
    if not os.path.isfile(sentences_path):
        if not os.path.isfile(os.path.join(data_dir, 'sentences.tar.bz2')):
            subprocess.run(
                "wget https://downloads.tatoeba.org/exports/sentences.tar.bz2 -P " + data_dir,
                shell=True)

        subprocess.run(
            "tar xvjC {0} -f {0}/sentences.tar.bz2".format(data_dir), shell=True)

    # download if links are not here
    links_path = os.path.join(data_dir, 'links.csv')
    if not os.path.isfile(links_path):
        if not os.path.isfile(os.path.join(data_dir, 'links.tar.bz2')):
            subprocess.run(
                "wget https://downloads.tatoeba.org/exports/links.tar.bz2 -P " + data_dir,
                shell=True)

        subprocess.run("tar xvjC {0} -f {0}/links.tar.bz2".format(data_dir), shell=True)

    # read all data
    sentences = pd.read_csv(sentences_path, names=['id', 'lang', 'text'], header=None, delimiter='\t')
    links = pd.read_csv(links_path, names=['sent_id', 'tran_id'], header=None, delimiter='\t')

    # extract source - target connected
    source_sentences = sentences[sentences.lang == source_lang]
    source_sentences = source_sentences.merge(links, left_on='id', right_on='sent_id')
    target_sentences = sentences[sentences.lang == target_lang]

    bilang_sentences = source_sentences.merge(target_sentences, left_on='tran_id',
                                              right_on='id',
                                              suffixes=[source_lang, target_lang])
    bilang_sentences = bilang_sentences[['text' + source_lang, 'text' + target_lang]]

    # save results
    file_name = os.path.join(data_dir, '{source}-{target}.csv'.format(source=source_lang, target=target_lang))
    bilang_sentences.to_csv(file_name, index=False, sep='\t')


In [ ]:
import pandas as pd
import os

source_lang = 'eng'
target_lang = 'tam'
data_dir = 'data/'

os.chdir('../')
corpus = pd.read_csv(os.path.join(data_dir, '{}-{}.csv'.format(source_lang, target_lang)), delimiter='\t')